In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import time
import datetime as datetime
from tqdm import tqdm

In [5]:
uso_df = pd.read_csv('USO_date_cleaned.csv')
datelist = pd.read_csv('datelist.csv')
dates = []
for i,val in datelist.iterrows():
    dates.append(val[1])

In [6]:
def oneDay(open_close, date, cash):
    # OPTIMIZE
    day = open_close[open_close['Date'] == date]
    
    rowiter = day.iterrows()
    for (i, row1), (j, row2) in zip(rowiter, rowiter):
        etfs_bought = 0
        if row1['Open'] < row1['Close']:
            etfs_bought = cash/row1['Close']
            cash -= etfs_bought * row1['Close']
            etfs_sold_price = etfs_bought * row2['Close']
            cash += etfs_sold_price
            # BUY FUNCTION
        else:
            etfs_sold = cash/row1['Close']
            cash += etfs_sold * row1['Close']
            etfs_bought_price = etfs_sold * row2['Close']
            cash -= etfs_bought_price
    #print("Bought: " + str(row1['Open'] < row1['Close']))
    return cash

Test for one day same day signal

In [7]:
# Set start and end time intervals, initial day for one time test
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"
day = "2012-01-03"
# Setting up dataframe with desired intervals
open = uso_df[uso_df['Time Interval'] == start_interval]
close = uso_df[uso_df['Time Interval'] == end_interval]
open_close = pd.concat([open, close])
open_close = open_close.sort_values(by='Date')
#
day = oneDay(open_close, day, 100)
day

100.53191489361703

Will perform oneDay for all days

In [10]:
def moreDays(dates, open_close):
    cash = 100
    for day in tqdm(dates):
        prev_cash = cash
        cash += oneDay(open_close, day, "09:30 - 10:00", "15:30 - 16:00", cash)
        cash -= prev_cash
    return cash

USE THIS IF YOU WANT TO TEST BASED ON CURRENT DAY SIGNAL

In [19]:
# Set start and end time intervals, initial day for one time test
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"

# Setting up dataframe with desired intervals
open = uso_df[uso_df['Time Interval'] == start_interval]
close = uso_df[uso_df['Time Interval'] == end_interval]
open_close = pd.concat([open, close])
open_close = open_close.sort_values(by='Date')
final = moreDays(dates, open_close)
print(final)

100%|██████████| 2552/2552 [00:03<00:00, 830.73it/s]

12.001626909365076


BENCHMARK BACK TESTER

In [21]:
def oneDayBenchmark(open_close, date, cash):
    day = open_close[open_close['Date'] == date]
    
    rowiter = day.iterrows()
    for (i, row1) in rowiter:
        etfs_bought = cash/row1['Open']
        cash -= etfs_bought * row1['Open']
        etfs_sold_price = etfs_bought * row1['Close']
        cash += etfs_sold_price

    return cash

99.97481108312341


Test the oneDayBenchmark on any one day

In [31]:
interval = "15:30 - 16:00"
day = "2012-01-03"
# Setting up dataframe with desired intervals
open_close = uso_df[uso_df['Time Interval'] == interval]
open_close = open_close.sort_values(by = 'Date')
final = oneDayBenchmark(open_close, day, 100)
print(final)

       Unnamed: 0  index  Time Interval   Close  Net Chg     Open      High  \
12             12     14  15:30 - 16:00  317.52  -0.0800  317.600  318.0000   
25             25     28  15:30 - 16:00  318.16  -0.0800  318.160  318.3200   
38             38     42  15:30 - 16:00  313.36  -0.5600  313.920  313.9200   
51             51     56  15:30 - 16:00  313.76   0.4800  313.352  313.9200   
64             64     70  15:30 - 16:00  312.64   0.5600  312.080  312.6400   
...           ...    ...            ...     ...      ...      ...       ...   
32824       33040    476  15:30 - 16:00   64.74   0.0300   64.710   64.8300   
32837       33053    490  15:30 - 16:00   65.53  -0.1508   65.670   65.7900   
32850       33066    504  15:30 - 16:00   66.24   0.1800   66.050   66.2750   
32863       33079    518  15:30 - 16:00   66.34   0.2100   66.120   66.4499   
32876       33092    532  15:30 - 16:00   65.80   0.0700   65.730   65.8500   

          Low  Tick Count    Volume        Date  We

Method moreDays to test benchmark on every day

In [23]:
def moreDaysBenchmark(dates, open_close):
    cash = 100
    for day in tqdm(dates):
        prev_cash = cash
        cash += oneDayBenchmark(open_close, day, cash)
        cash -= prev_cash
    return cash

In [26]:
# Set up interval here
interval = "15:30 - 16:00"

# Setting up dataframe with desired intervals
open_close = uso_df[uso_df['Time Interval'] == interval]
open_close = open_close.sort_values(by = 'Date')
final = moreDaysBenchmark(dates, open_close)
print(final)

100%|██████████| 2552/2552 [00:02<00:00, 1054.54it/s]

127.26732073008941


In [66]:
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"
# cash = 100
day = "2012-01-30"

def oneDay2(open_close, i1, i2, cash):
    # OPTIMIZE
    day = open_close.iloc[i1:i2]

    rowiter = day.iterrows()
    for (i, row1), (j, row2), (k, row3) in zip(rowiter, rowiter, rowiter):
        etfs_bought = 0
        p1 = row2['Close']
        p0 = row1['Close']
        r1 = math.log((p1/p0), 10)
        if r1 >= 0:
            etfs_bought = int(cash/row3['Open'])
            cash -= etfs_bought * row3['Open']
            etfs_sold_price = etfs_bought * row3['Close']
            cash += etfs_sold_price
            # BUY FUNCTION
        else:
            etfs_sold = int(cash/row3['Open'])
            cash += etfs_sold * row3['Open']
            etfs_bought_price = etfs_sold * row3['Close']
            cash -= etfs_bought_price
    #print("Bought: " + str(row1['Open'] < row1['Close']))

    return cash

In [67]:
# Set desired Intervals
early_interval = "09:30 - 10:00" # On current day
late_interval = "15:30 - 16:00" # On previous day

#
open = uso_df[uso_df['Time Interval'] == start_interval]
close = uso_df[uso_df['Time Interval'] == end_interval]
open_close = pd.concat([open, close])
open_close = open_close.sort_index()
open_close = open_close.sort_values(by='Date')
open_close = open_close.drop(['Unnamed: 0', 'index'], axis = 1)
open_close = open_close.reset_index(drop = True)

i1 = 1
i2 = 4
day = oneDay2(open_close, i1,i2, 1000)
day

1000.0

In [68]:
i1 = 1
i2 = 4
cash=1000
while i2 < 5087:
    prev_cash = cash
    cash += oneDay2(open_close, i1, i2, cash)
    cash -= prev_cash
    i1 += 2
    i2 += 2


print(cash)

1753.5855999999794
